Project work part II

Motivated by the sequential methods we discussed in class, our goal here is to design (7
points) and implement (7 points) a new method for producing sequential group
recommendations. Also, provide detailed explanations and clarifications about why the
method you propose works well for the case of sequential group recommendations (6
points). Prepare also a short presentation (about 5 slides) to show how your method
works (5 points). 

In [ ]:

import pandas as pd
import numpy as np
from collections import defaultdict

In [2]:
links = pd.read_csv('ml-latest-small/links.csv')
movies = pd.read_csv('ml-latest-small/movies.csv')
ratings = pd.read_csv('ml-latest-small/ratings.csv')
tags = pd.read_csv('ml-latest-small/tags.csv')

print("Links Dataset:")
display(links.head())

print("\nMovies Dataset:")
display(movies.head())

print("\nRatings Dataset:")
display(ratings.head())

print("\nTags Dataset:")
display(tags.head())

rating_count = ratings.shape[0]
print(f"\nTotal number of ratings: {rating_count}")

Links Dataset:


,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0



Movies Dataset:


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy



Ratings Dataset:


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931



Tags Dataset:


,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200



Total number of ratings: 100836


In [ ]:
group_users = [1, 2, 3, 4, 5]

# Satisfaction scores for the users (initially all unsatisfied)
satisfaction_scores = {user: 0.0 for user in group_users}

# Keep track of movies already recommended for each user
user_history = {user: set() for user in group_users}

# Function to recommend items to a single user based on their ratings
def recommend_for_user(user_id, num_recommendations=5):
    # Get movies rated highly by the user
    user_ratings = ratings[ratings['userId'] == user_id]
    high_rated_movies = user_ratings[user_ratings['rating'] >= 4.0]
    
    # Get genres of highly-rated movies
    high_rated_movie_ids = high_rated_movies['movieId'].unique()
    high_rated_genres = movies[movies['movieId'].isin(high_rated_movie_ids)]['genres']
    genre_set = set([genre for genres in high_rated_genres for genre in genres])
    
    # Recommend movies with similar genres not yet rated or recommended by the user
    candidate_movies = movies[~movies['movieId'].isin(user_ratings['movieId'])]
    candidate_movies = candidate_movies.copy()  # Avoid SettingWithCopyWarning
    candidate_movies['genre_match'] = candidate_movies['genres'].apply(lambda x: len(set(x) & genre_set))
    
    # Exclude already recommended movies
    already_recommended = user_history[user_id]
    candidate_movies = candidate_movies[~candidate_movies['movieId'].isin(already_recommended)]
    
    # Sort by genre match and select top recommendations
    recommended_movies = candidate_movies.sort_values(by='genre_match', ascending=False).head(num_recommendations)
    recommended_movie_ids = recommended_movies['movieId'].tolist()
    
    # Update user history
    user_history[user_id].update(recommended_movie_ids)
    
    return recommended_movie_ids

# Function to aggregate individual recommendations into group recommendations
def aggregate_group_recommendations(user_recommendations, weights):
    recommendation_scores = defaultdict(float)
    for user, recommendations in user_recommendations.items():
        weight = weights[user]
        for movie_id in recommendations:
            recommendation_scores[movie_id] += weight
    # Sort movies by aggregated score
    sorted_recommendations = sorted(recommendation_scores.items(), key=lambda x: -x[1])
    return [movie[0] for movie in sorted_recommendations]


# Function to simulate multiple iterations with dynamic recommendations
num_iterations = 5
for iteration in range(1, num_iterations + 1):
    print(f"\nIteration {iteration}:")
    
    # Single user recommendations
    user_recommendations = {}
    for user in group_users:
        user_recommendations[user] = recommend_for_user(user)
        print(f"User {user} recommendations: {user_recommendations[user]}")
    
    # Calculate weights for group aggregation
    weights = {user: 1.0 + (1.0 - satisfaction_scores[user]) for user in group_users}
    
    # Group recommendations
    group_recommendations = aggregate_group_recommendations(user_recommendations, weights)
    print(f"Group recommendations: {group_recommendations}")
    
    # Evaluate satisfaction scores
    for user in group_users:
        # Check how many of the user's recommendations are in the group recommendations
        matched_recommendations = set(user_recommendations[user]) & set(group_recommendations)
        if len(matched_recommendations) >= 3:  # At least 3 recommendations in group recommendations
            satisfaction_scores[user] = min(1.0, satisfaction_scores[user] + 0.5)  # Increase satisfaction
        else:
            satisfaction_scores[user] = max(0.0, satisfaction_scores[user] - 0.5)  # Decrease satisfaction
    
    # Print satisfaction scores
    print(f"Satisfaction scores: {satisfaction_scores}")


Iteration 1:
User 1 recommendations: [81132, 6902, 71999, 26701, 32031]
User 2 recommendations: [6902, 81132, 32031, 71999, 85261]
User 3 recommendations: [6902, 81132, 32031, 148775, 164226]
User 4 recommendations: [81132, 6902, 71999, 148775, 26701]
User 5 recommendations: [81132, 32031, 81847, 6902, 78637]
Group recommendations: [81132, 6902, 32031, 71999, 26701, 148775, 85261, 164226, 81847, 78637]
Satisfaction scores: {1: 0.5, 2: 0.5, 3: 0.5, 4: 0.5, 5: 0.5}

Iteration 2:
User 1 recommendations: [164226, 2367, 148775, 56152, 85261]
User 2 recommendations: [148775, 60684, 81847, 2367, 164226]
User 3 recommendations: [71999, 26701, 26590, 117646, 72165]
User 4 recommendations: [2367, 32031, 164226, 79132, 130520]
User 5 recommendations: [98243, 58105, 85261, 71745, 71999]
Group recommendations: [164226, 2367, 148775, 85261, 71999, 56152, 60684, 81847, 26701, 26590, 117646, 72165, 32031, 79132, 130520, 98243, 58105, 71745]
Satisfaction scores: {1: 1.0, 2: 1.0, 3: 1.0, 4: 1.0, 5: 1.0

Tässä vasta pohdintaan, vaihdetaan näitä sitten englanniksi jos päädytään tähän ratkaisuun. 

Valitsemme yksinkertaisen Markovin ketjuihin perustuvan menetelmän, joka ottaa huomioon ryhmän katseluhistorian ja soveltaa sitä sekventiaalisiin suosituksiin. Lisäksi yhdistämme siihen ryhmäkohtaista aggregointia, joka mukautuu ryhmän mieltymysten mukaan.

Menetelmän ydin:
Siirtymätodennäköisyydet:

Lasketaan siirtymätodennäköisyydet genrejen välillä käyttäen käyttäjäkohtaisia katseluhistorioita. Näitä siirtymiä käytetään ennustamaan seuraavia genrejä.
Ryhmäaggregointi:

Käytetään keskiarvomenetelmää: Yhdistetään yksittäisten käyttäjien suositukset keskiarvoistamalla genren siirtymätodennäköisyydet.
Käytetään myös vähimmän tyytyväisyyden menetelmää (least misery): Suositellaan vain genrejä, jotka kaikki ryhmän jäsenet hyväksyisivät. Tämä toimii kompromissiratkaisuna.
Sekventiaalisuuden huomioiminen:

Käytetään viimeisintä katsottua genreä tai viimeisimpiä genrejä ennustamaan ryhmän seuraavaa valintaa. Näin menetelmä mukautuu ajan mukana ryhmän tottumuksiin.


In [3]:
import pandas as pd
import numpy as np
from collections import defaultdict


# Erotellaan genre ensimmäisenä olevaksi genrekentäksi (jos genret eroteltu "|"-merkeillä)
movies['primary_genre'] = movies['genres'].apply(lambda x: x.split('|')[0] if isinstance(x, str) else None)

# Yhdistetään datasetit movieId:n perusteella
data = pd.merge(ratings, movies[['movieId', 'primary_genre']], on='movieId')

# Tarkastetaan yhdistetty data
print(data.head())

# Nyt jokaisella rivillä on userId, movieId, rating, timestamp ja primary_genre

# Lasketaan genre-siirtymät käyttäjäkohtaisesti
transition_counts = defaultdict(lambda: defaultdict(int))

# Käydään läpi käyttäjäkohtaiset genret aikajärjestyksessä
users = data['userId'].unique()
for user in users:
    user_data = data[data['userId'] == user].sort_values(by="timestamp")
    genres = user_data['primary_genre'].tolist()
    for i in range(len(genres) - 1):
        current_genre = genres[i]
        next_genre = genres[i + 1]
        transition_counts[current_genre][next_genre] += 1

# Muutetaan siirtymätodennäköisyyksiksi
transition_probabilities = {}
for genre, transitions in transition_counts.items():
    total_transitions = sum(transitions.values())
    transition_probabilities[genre] = {next_genre: count / total_transitions for next_genre, count in transitions.items()}

# Tulostetaan siirtymätodennäköisyydet
print("Transition probabilities:")
for genre, probs in transition_probabilities.items():
    print(f"{genre}: {probs}")

# Suositustoiminto: suosittele seuraava genre
def recommend_next_genre(current_genre):
    next_genre_probs = transition_probabilities.get(current_genre, {})
    if not next_genre_probs:
        return np.random.choice(list(transition_probabilities.keys()))  # Palauta satunnainen genre, jos ei ole siirtymätietoa
    genres = list(next_genre_probs.keys())
    probs = list(next_genre_probs.values())
    return np.random.choice(genres, p=probs)

# Testataan suositusta
current_genre = "Comedy"
next_genre = recommend_next_genre(current_genre)
print(f"Next recommended genre after {current_genre}: {next_genre}")


   userId  movieId  rating  timestamp primary_genre
0       1        1     4.0  964982703     Adventure
1       1        3     4.0  964981247        Comedy
2       1        6     4.0  964982224        Action
3       1       47     5.0  964983815       Mystery
4       1       50     5.0  964982931         Crime
Transition probabilities:
Action: {'Comedy': 0.18136772330511255, 'Action': 0.4380127321651244, 'Drama': 0.13057032224191115, 'Children': 0.014930760648421606, 'Adventure': 0.10645140119446085, 'Horror': 0.02077180547351841, 'Thriller': 0.0064645271378880355, 'Animation': 0.019360766555096148, 'Crime': 0.055522740696987594, 'Documentary': 0.00534882194657741, 'Sci-Fi': 0.0038721533110192296, 'Western': 0.0010500754741747064, 'Mystery': 0.010467939883179104, 'Film-Noir': 0.000459408019951434, 'Musical': 0.0009844457570387872, 'Romance': 0.0008531863227669488, '(no genres listed)': 0.0004265931613834744, 'War': 0.00016407429283979785, 'Fantasy': 0.0029205224125484017}
Comedy: {'Ani

Miksi tämä menetelmä toimii?
Sekventiaalisuuden huomioiminen:

Markovin ketjut ottavat suoraan huomioon ryhmän aiemmat valinnat ja niiden väliset siirtymät, mikä tekee suosituksista mukautuvia.
Sekventiaalisuus mahdollistaa ryhmän mieltymysten dynaamisen seuraamisen ajan myötä.
Ryhmäaggregoinnin käyttö:

Keskiarvomenetelmä varmistaa, että kaikkien ryhmän jäsenten mieltymykset huomioidaan tasapuolisesti.
Vähimmän tyytyväisyyden menetelmä varmistaa, että suositukset eivät jätä yksittäistä ryhmän jäsentä täysin tyytymättömäksi.
Helppo toteuttaa ja laajentaa:

Markovin ketjut ovat suoraviivaisia ja laskettavissa ilman raskasta laskentaa.
Menetelmää voidaan laajentaa helposti lisäämällä esimerkiksi ajallisia painotuksia, kuten antamalla enemmän merkitystä tuoreimmille valinnoille.
